In [1]:
import os
import sys
import warnings
import json 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore")

In [2]:
# Current code folder
current_path = os.getcwd()

# Get the assignment1 folder
if "code" in current_path:
    assignment_dir = os.path.dirname(current_path)
else:
    assignment_dir = current_path

# Location folders
data_in = os.path.join(assignment_dir, "data", "raw/")
data_out = os.path.join(assignment_dir, "data", "clean/")
output = os.path.join(assignment_dir, "output/")

# Data Cleaning Function

The following function contains all data cleaning steps. It will be applied to each dataset.

In [3]:
def clean_dataset(dataset):
    """
    Clean an Airbnb dataset.
    
    Parameters:
        dataset: str - e.g. 'madrid_25q1', 'madrid_25q2', 'valencia_25q1'
    """
    
    input_file = f"listings_{dataset}.csv"
    output_file = f"airbnb_{dataset}_clean.csv"
    
    print("\n" + "="*60)
    print(f"CLEANING: {dataset}")
    print("="*60)
    
    # --------------------------------------------------------
    # READ DATA
    # --------------------------------------------------------
    df = pd.read_csv(data_in + input_file)
    print(f"\nLoaded: {df.shape[0]} rows, {df.shape[1]} columns")
    
    # --------------------------------------------------------
    # CLEAN PRICE (TARGET)
    # --------------------------------------------------------
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
    df = df[(df['price'].notna()) & (df['price'] > 0) & (df['price'] < 1000)]
    print(f"After price filter: {df.shape[0]} rows")
    
    df["ln_price"] = np.log(df.price)
    
    # --------------------------------------------------------
    # PROPERTY TYPE - simplify to Apartment/House/Other
    # --------------------------------------------------------
    def simplify_property(pt):
        if pd.isna(pt):
            return 'Other'
        pt = pt.lower()
        if any(x in pt for x in ['apartment', 'rental unit', 'condo', 'loft', 'serviced']):
            return 'Apartment'
        if any(x in pt for x in ['house', 'home', 'townhouse', 'villa', 'cottage', 'bungalow', 'cabin']):
            return 'House'
        return 'Other'
    
    df['f_property_type'] = df['property_type'].apply(simplify_property).astype('category')
    
    # Exclude hotels and hostels (for better comparability, only dropping a few observations)
    df = df[~df['property_type'].str.lower().str.contains('hotel|hostel', na=False)]
    print(f"After excluding hotels/hostels: {df.shape[0]} rows")
    
    # --------------------------------------------------------
    # FACTOR VARIABLES
    # --------------------------------------------------------
    df = df[~df['room_type'].str.lower().str.contains('hotel|hostel', na=False)]
    df['f_room_type'] = df['room_type'].astype('category')
    
    # Keep only neighbourhoods with at least 30 observations (important for cross validation, rare neighbourhoods are problematic)
    # Later on I decided to exlude neigbourhood variable: external validation using Valencia does not work
    neighbourhood_counts = df['neighbourhood_cleansed'].value_counts()
    keep_neighbourhoods = neighbourhood_counts[neighbourhood_counts >= 30].index
    df = df[df['neighbourhood_cleansed'].isin(keep_neighbourhoods)]
    df['f_neighbourhood_cleansed'] = df['neighbourhood_cleansed'].astype('category')
    
    # --------------------------------------------------------
    # NUMERICAL VARIABLES
    # --------------------------------------------------------
    df['n_accommodates'] = df['accommodates'].astype(float)
    df['n_bedrooms'] = df['bedrooms'].astype(float)
    df['n_beds'] = df['beds'].astype(float)
    df['n_bathrooms'] = df['bathrooms'].astype(float)
    
    # Fill missing bathrooms from bathrooms_text
    mask = df['n_bathrooms'].isna() & df['bathrooms_text'].notna()
    df.loc[mask, 'n_bathrooms'] = df.loc[mask, 'bathrooms_text'].str.extract(r'(\d+\.?\d*)')[0].astype(float)
    
    # Other numerical variables
    df['n_number_of_reviews'] = df['number_of_reviews'].astype(float)
    df['n_reviews_per_month'] = df['reviews_per_month'].astype(float)
    df['n_review_scores_rating'] = df['review_scores_rating'].astype(float)
    df['n_minimum_nights'] = df['minimum_nights'].astype(float)
    df['n_availability_365'] = df['availability_365'].astype(float)
    df['n_host_listings_count'] = df['host_listings_count'].astype(float)
    
    # Host response rate
    df['p_host_response_rate'] = df['host_response_rate'].str.replace('%', '').astype(float)
    
    # --------------------------------------------------------
    # DUMMY VARIABLES (binary)
    # --------------------------------------------------------
    df['d_host_is_superhost'] = (df['host_is_superhost'] == 't').astype(int)
    df['d_instant_bookable'] = (df['instant_bookable'] == 't').astype(int)
    df['d_host_identity_verified'] = (df['host_identity_verified'] == 't').astype(int)
    
    # --------------------------------------------------------
    # DAYS SINCE FIRST REVIEW
    # --------------------------------------------------------
    df['n_days_since'] = (pd.to_datetime(df['last_scraped']) - pd.to_datetime(df['first_review'])).dt.days
    
    # --------------------------------------------------------
    # AMENITIES EXTRACTION
    # --------------------------------------------------------
    def parse_amenities(x):
        if pd.isna(x): return []
        try: return json.loads(x)
        except: return re.findall(r'"([^"]*)"', x)
    
    df['amenities_list'] = df['amenities'].apply(parse_amenities)
    
    amenities_to_extract = [
        'Wifi', 'Air conditioning', 'Heating', 'Kitchen', 'Washer', 'Dryer',
        'TV', 'Hair dryer', 'Iron', 'Elevator', 'Free parking', 'Pool', 'Dishes and silverware',
        'Hot tub', 'Gym', 'Self check-in', 'Smoking allowed', 'Pets allowed', 'Hot water',
        'Coffee maker', 'Dishwasher', 'Microwave', 'Refrigerator', 'Balcony',
        'Fire extinguisher', 'Smoke alarm', 'Carbon monoxide alarm', 
        'Dedicated workspace', 'Essentials', 'Hangers', 'Bed linens'
    ]
    
    for amenity in amenities_to_extract:
        col_name = 'd_' + re.sub(r'[^a-z0-9]', '', amenity.lower())
        df[col_name] = df['amenities_list'].apply(
            lambda x: int(any(amenity.lower() in a.lower() for a in x))
        )
    
    df['n_amenities_count'] = df['amenities_list'].apply(len)
    
    # --------------------------------------------------------
    # DERIVED FEATURES
    # --------------------------------------------------------
    df['n_accommodates2'] = df['n_accommodates'] ** 2
    df['ln_accommodates'] = np.log(df['n_accommodates'] + 1)
    df['ln_accommodates2'] = df['ln_accommodates'] ** 2
    df['ln_beds'] = np.log(df['n_beds'].fillna(1) + 1)
    df['ln_number_of_reviews'] = np.log(df['n_number_of_reviews'] + 1)
    
    # --------------------------------------------------------
    # CATEGORICAL BINS
    # --------------------------------------------------------
    df['f_bathroom'] = pd.cut(df['n_bathrooms'].fillna(1), bins=[-0.1, 0.9, 1.9, 100], labels=[0, 1, 2])
    df['f_number_of_reviews'] = pd.cut(df['n_number_of_reviews'], bins=[-0.1, 0.9, 10, 50, 10000], labels=[0, 1, 2, 3])
    df['f_minimum_nights'] = pd.cut(df['n_minimum_nights'], bins=[0, 1, 2, 7, 10000], labels=[1, 2, 3, 4])
    
    # --------------------------------------------------------
    # HANDLE MISSING VALUES
    # --------------------------------------------------------
    # Impute missign with median
    for col in ['n_bathrooms', 'n_beds', 'n_bedrooms']:
        df[col] = df[col].fillna(df[col].median())
    
    for col in ['n_days_since', 'n_review_scores_rating', 'n_reviews_per_month', 'p_host_response_rate']:
        flag_name = 'flag_' + col.replace('n_', '').replace('p_', '')
        df[flag_name] = df[col].isna().astype(int)
        df[col] = df[col].fillna(df[col].median())
    
    # Recalculate log features after imputation
    df['ln_beds'] = np.log(df['n_beds'] + 1)
    df['ln_days_since'] = np.log(df['n_days_since'] + 1)
    df['ln_days_since2'] = df['ln_days_since'] ** 2
    df['ln_days_since3'] = df['ln_days_since'] ** 3
    df['ln_review_scores_rating'] = np.log(df['n_review_scores_rating'] + 0.01)
    
    # Replace infinities with nan
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)
    df[numeric_cols] = df[numeric_cols].fillna(0)
    
    # Fill categorical NAs with mode (most common category)
    categorical_cols = df.select_dtypes(include=['category']).columns
    for col in categorical_cols:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].mode()[0])
    
    # --------------------------------------------------------
    # SELECT FINAL COLUMNS
    # --------------------------------------------------------
    keep_cols = ['id', 'price', 'latitude', 'longitude']
    keep_cols += [c for c in df.columns if c.startswith(('f_', 'n_', 'd_', 'ln_', 'p_', 'flag_'))]
    keep_cols = list(dict.fromkeys(keep_cols))
    
    data = df[keep_cols].copy()
    
    # --------------------------------------------------------
    # SAVE
    # --------------------------------------------------------
    data.to_csv(data_out + output_file, index=False)
    
    print(f"\nFinal shape: {data.shape}")
    print(f"Saved to: {data_out}{output_file}")
    print(f"Price: mean={data['price'].mean():.0f}, median={data['price'].median():.0f}")
    
    return data

## Data Cleaning Steps

### 1. Price (Target Variable)
- Remove dollar signs and commas
- Filter to valid prices: >0 and <1000
- Create log transformation

### 2. Property Type
- Simplify to three categories: Apartment, House, Other
- Exclude hotels and hostels

### 3. Factor Variables
- Room type: converted to category
- Neighbourhood: keep only those with 30+ observations (for cross-validation stability). Eventually dropped from final set for external validity reasons.

### 4. Numerical Variables
- Accommodates, bedrooms, beds, bathrooms
- Reviews: count, rating, per month
- Host response rate (converted from percentage string)

### 5. Dummy Variables
- Superhost status
- Instant bookable
- Host identity verified

### 6. Amenities
- Parse JSON/string amenities list
- Extract 31 key amenities as binary dummies
- Count total amenities

### 7. Derived Features
- Polynomial terms: accommodates², ln(accommodates)
- Log transformations for skewed variables
- Categorical bins for bathrooms, reviews, minimum nights

### 8. Missing Values
- Numerical: impute with median
- Categorical: impute with mode
- Create flags for imputed values (days_since, review_scores, etc.)
- Replace infinities with 0

# Run Cleaning on All Datasets

Process all three datasets:
- **Madrid Q1 2025**: Training data
- **Madrid Q2 2025**: Temporal validity
- **Valencia Q1 2025**: External validity

In [4]:
# Clean all three datasets
datasets = ["madrid_25q1", "madrid_25q2", "valencia_25q1"]

cleaned_data = {}
for dataset in datasets:
    cleaned_data[dataset] = clean_dataset(dataset)

print("\n" + "="*60)
print("ALL DATASETS CLEANED SUCCESSFULLY")
print("="*60)


CLEANING: madrid_25q1

Loaded: 25288 rows, 79 columns
After price filter: 19179 rows
After excluding hotels/hostels: 18688 rows

Final shape: (18169, 71)
Saved to: C:\Users\borik\Desktop\Data-Analysis-3\assignment1\data\clean/airbnb_madrid_25q1_clean.csv
Price: mean=121, median=98

CLEANING: madrid_25q2

Loaded: 26004 rows, 79 columns
After price filter: 19969 rows
After excluding hotels/hostels: 19481 rows

Final shape: (19035, 71)
Saved to: C:\Users\borik\Desktop\Data-Analysis-3\assignment1\data\clean/airbnb_madrid_25q2_clean.csv
Price: mean=124, median=101

CLEANING: valencia_25q1

Loaded: 8847 rows, 79 columns
After price filter: 8081 rows
After excluding hotels/hostels: 7945 rows

Final shape: (7606, 71)
Saved to: C:\Users\borik\Desktop\Data-Analysis-3\assignment1\data\clean/airbnb_valencia_25q1_clean.csv
Price: mean=131, median=114

ALL DATASETS CLEANED SUCCESSFULLY


In [5]:
# Summary comparison
print("\nDataset Comparison:")
print("-" * 50)
for name, df in cleaned_data.items():
    print(f"{name}: {df.shape[0]:,} obs, {df.shape[1]} vars, "
          f"mean price=€{df['price'].mean():.0f}")


Dataset Comparison:
--------------------------------------------------
madrid_25q1: 18,169 obs, 71 vars, mean price=€121
madrid_25q2: 19,035 obs, 71 vars, mean price=€124
valencia_25q1: 7,606 obs, 71 vars, mean price=€131
